# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 17 2022 1:48PM,239869,10,6598292,Acute Outpatient Solutions,Released
1,Jun 17 2022 1:47PM,239868,10,0085775774,ISDIN Corporation,Released
2,Jun 17 2022 1:47PM,239868,10,0085778316,ISDIN Corporation,Released
3,Jun 17 2022 1:47PM,239868,10,0085778315,ISDIN Corporation,Released
4,Jun 17 2022 1:47PM,239868,10,0085778317,ISDIN Corporation,Released
5,Jun 17 2022 1:47PM,239868,10,0085778341,ISDIN Corporation,Released
6,Jun 17 2022 1:47PM,239868,10,0085778469,ISDIN Corporation,Released
7,Jun 17 2022 1:47PM,239868,10,0085778470,ISDIN Corporation,Released
8,Jun 17 2022 1:47PM,239868,10,0085778485,ISDIN Corporation,Released
9,Jun 17 2022 1:47PM,239868,10,0085778480,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,239865,Released,34
21,239866,Released,5
22,239867,Released,1
23,239868,Released,56
24,239869,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239865,NaN,NaN,34.0
239866,NaN,NaN,5.0
239867,NaN,NaN,1.0
239868,NaN,NaN,56.0
239869,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239824,1.0,13.0,5.0
239828,0.0,2.0,0.0
239830,0.0,0.0,1.0
239835,0.0,0.0,1.0
239837,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239824,1,13,5
239828,0,2,0
239830,0,0,1
239835,0,0,1
239837,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239824,1,13,5
1,239828,0,2,0
2,239830,0,0,1
3,239835,0,0,1
4,239837,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239824,1,13,5
1,239828,,2,
2,239830,,,1
3,239835,,,1
4,239837,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 17 2022 1:48PM,239869,10,Acute Outpatient Solutions
1,Jun 17 2022 1:47PM,239868,10,ISDIN Corporation
57,Jun 17 2022 1:39PM,239867,10,ISDIN Corporation
58,Jun 17 2022 1:37PM,239866,10,ISDIN Corporation
63,Jun 17 2022 1:37PM,239865,10,ISDIN Corporation
97,Jun 17 2022 1:34PM,239864,19,Eli Lilly and Company
98,Jun 17 2022 1:27PM,239863,20,Alumier Labs Inc.
116,Jun 17 2022 1:06PM,239861,12,Hush Hush
120,Jun 17 2022 1:05PM,239860,12,Hush Hush
121,Jun 17 2022 12:54PM,239859,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 17 2022 1:48PM,239869,10,Acute Outpatient Solutions,,,1
1,Jun 17 2022 1:47PM,239868,10,ISDIN Corporation,,,56
2,Jun 17 2022 1:39PM,239867,10,ISDIN Corporation,,,1
3,Jun 17 2022 1:37PM,239866,10,ISDIN Corporation,,,5
4,Jun 17 2022 1:37PM,239865,10,ISDIN Corporation,,,34
5,Jun 17 2022 1:34PM,239864,19,Eli Lilly and Company,,,1
6,Jun 17 2022 1:27PM,239863,20,Alumier Labs Inc.,,,18
7,Jun 17 2022 1:06PM,239861,12,Hush Hush,,,4
8,Jun 17 2022 1:05PM,239860,12,Hush Hush,,,1
9,Jun 17 2022 12:54PM,239859,10,"Citieffe, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 1:48PM,239869,10,Acute Outpatient Solutions,1,,
1,Jun 17 2022 1:47PM,239868,10,ISDIN Corporation,56,,
2,Jun 17 2022 1:39PM,239867,10,ISDIN Corporation,1,,
3,Jun 17 2022 1:37PM,239866,10,ISDIN Corporation,5,,
4,Jun 17 2022 1:37PM,239865,10,ISDIN Corporation,34,,
5,Jun 17 2022 1:34PM,239864,19,Eli Lilly and Company,1,,
6,Jun 17 2022 1:27PM,239863,20,Alumier Labs Inc.,18,,
7,Jun 17 2022 1:06PM,239861,12,Hush Hush,4,,
8,Jun 17 2022 1:05PM,239860,12,Hush Hush,1,,
9,Jun 17 2022 12:54PM,239859,10,"Citieffe, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 1:48PM,239869,10,Acute Outpatient Solutions,1,,
1,Jun 17 2022 1:47PM,239868,10,ISDIN Corporation,56,,
2,Jun 17 2022 1:39PM,239867,10,ISDIN Corporation,1,,
3,Jun 17 2022 1:37PM,239866,10,ISDIN Corporation,5,,
4,Jun 17 2022 1:37PM,239865,10,ISDIN Corporation,34,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 1:48PM,239869,10,Acute Outpatient Solutions,1.0,NaN,NaN
1,Jun 17 2022 1:47PM,239868,10,ISDIN Corporation,56.0,NaN,NaN
2,Jun 17 2022 1:39PM,239867,10,ISDIN Corporation,1.0,NaN,NaN
3,Jun 17 2022 1:37PM,239866,10,ISDIN Corporation,5.0,NaN,NaN
4,Jun 17 2022 1:37PM,239865,10,ISDIN Corporation,34.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 1:48PM,239869,10,Acute Outpatient Solutions,1.0,0.0,0.0
1,Jun 17 2022 1:47PM,239868,10,ISDIN Corporation,56.0,0.0,0.0
2,Jun 17 2022 1:39PM,239867,10,ISDIN Corporation,1.0,0.0,0.0
3,Jun 17 2022 1:37PM,239866,10,ISDIN Corporation,5.0,0.0,0.0
4,Jun 17 2022 1:37PM,239865,10,ISDIN Corporation,34.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2878265,102.0,2.0,0.0
12,719549,5.0,2.0,0.0
18,239844,1.0,0.0,0.0
19,479722,2.0,0.0,0.0
20,959362,52.0,37.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2878265,102.0,2.0,0.0
1,12,719549,5.0,2.0,0.0
2,18,239844,1.0,0.0,0.0
3,19,479722,2.0,0.0,0.0
4,20,959362,52.0,37.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,102.0,2.0,0.0
1,12,5.0,2.0,0.0
2,18,1.0,0.0,0.0
3,19,2.0,0.0,0.0
4,20,52.0,37.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,102.0
1,12,Released,5.0
2,18,Released,1.0
3,19,Released,2.0
4,20,Released,52.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,18,19,20
Status,,,,,
Completed,0.0,0.0,0.0,0.0,1.0
Executing,2.0,2.0,0.0,0.0,37.0
Released,102.0,5.0,1.0,2.0,52.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,18,19,20
0,Completed,0.0,0.0,0.0,0.0,1.0
1,Executing,2.0,2.0,0.0,0.0,37.0
2,Released,102.0,5.0,1.0,2.0,52.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,18,19,20
0,Completed,0.0,0.0,0.0,0.0,1.0
1,Executing,2.0,2.0,0.0,0.0,37.0
2,Released,102.0,5.0,1.0,2.0,52.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()